# Final Project: Vehicle Rounting Problem

In [1]:
import numpy as np
#np.set_printoptions(threshold=np.nan)
#import seaborn as sns
from matplotlib import pyplot as plt

## 1. Import Data

In [2]:
# VRP1
# import distance matrix
filepath = 'VRP1/distance.txt'
with open(filepath, 'r') as t:
    distance_matrix = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        distance_matrix.append(y)
        
    t.close()
    distance_matrix = np.array(distance_matrix)

# import demand matrix
filepath = 'VRP1/demand.txt'
with open(filepath, 'r') as t:
    demand_matrix = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        demand_matrix.append(y)
        
    t.close()
    demand_matrix = np.array(demand_matrix)

# import capacity matrix
filepath = 'VRP1/capacity.txt'
with open(filepath, 'r') as t:
    capacity_matrix = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        capacity_matrix.append(y)
        
    t.close()
    capacity_matrix = np.array(capacity_matrix)

# VRP1
# import distance matrix
filepath = 'VRP2/distance.txt'
with open(filepath, 'r') as t:
    distance_matrix_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        distance_matrix_2.append(y)
        
    t.close()
    distance_matrix_2 = np.array(distance_matrix_2)

# import demand matrix
filepath = 'VRP2/demand.txt'
with open(filepath, 'r') as t:
    demand_matrix_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        demand_matrix_2.append(y)
        
    t.close()
    demand_matrix_2 = np.array(demand_matrix)

# import capacity matrix
filepath = 'VRP2/capacity.txt'
with open(filepath, 'r') as t:
    capacity_matrix_2 = []
    for line in t.readlines():
        y = [int(value) for value in line.split()]
        capacity_matrix_2.append(y)
        
    t.close()
    capacity_matrix_2 = np.array(capacity_matrix)

In [3]:
# information about the distance matrix for VRP1
print("Information about distance matric for VRP1 \n# columns: ", len(distance_matrix[0]))
print("# lines: ", len(distance_matrix))
asymmetric = (distance_matrix != distance_matrix.T)
print(np.unique(asymmetric))
print("Since we also get False values while comparing the matrix with its transposed, the matrix is asymmetric. \n")

# information about the distance matrix for VRP2
print("Information about distance matric for VRP2 \n# columns: ", len(distance_matrix_2[0]))
print("# lines: ", len(distance_matrix_2))
asymmetric_2 = (distance_matrix_2 != distance_matrix_2.T)
print(np.unique(asymmetric_2))
print("Since we also get False values while comparing the matrix with its transposed, the matrix is asymmetric.")

Information about distance matric for VRP1 
# columns:  101
# lines:  101
[False  True]
Since we also get False values while comparing the matrix with its transposed, the matrix is asymmetric. 

Information about distance matric for VRP2 
# columns:  101
# lines:  101
[False  True]
Since we also get False values while comparing the matrix with its transposed, the matrix is asymmetric.


In [4]:
#heatmap
ax = sns.heatmap(distance_matrix, center=0)

NameError: name 'sns' is not defined

The heatmap again illustrates the fact that we have to deal with an asymmetric distance matrix. In order to apply the k-means algorithm, we need a symmetric matrix. Therefore we need to make our distances symmetric be taking the mean between the distances (if they are different). 

In [5]:
# CASE 1: distance_matrix 
cleaned_distance_matrix = np.copy(distance_matrix)
for i in range(len(asymmetric)):
    for j in range(i+1, len(asymmetric[0])):
        if asymmetric[i,j]:
            cleaned_distance = (distance_matrix[i,j] + distance_matrix[j,i]) // 2
            cleaned_distance_matrix[i,j] = cleaned_distance
            cleaned_distance_matrix[j,i] = cleaned_distance
            
# CASE 2: distance_matrix_2 
cleaned_distance_matrix_2 = np.copy(distance_matrix_2)
for i in range(len(asymmetric_2)):
    for j in range(i+1, len(asymmetric_2[0])):
        if asymmetric_2[i,j]:
            cleaned_distance = (distance_matrix_2[i,j] + distance_matrix_2[j,i]) // 2
            cleaned_distance_matrix_2[i,j] = cleaned_distance
            cleaned_distance_matrix_2[j,i] = cleaned_distance

In [6]:
# Demand matrix has 100 entries
demand_matrix.shape

# Note that the matrix is 101x101, i.e. that the depot is set at (0)

(1, 100)

## Algorithm

1. Clean matrix, i.e. make the matrix symmetric. Take mean of two different values.
2. K-means clustering to 100-demand-clusters. See Vanessa's paper how to do it. 
3. Merging: first calculate the demand-distance-ratio for the existing 100-demand-clusters. Then merge 3 100-demand-clusters with K-means clustering into one (do that until all are merged). Calculate demand-distance-ratio of the 300-clusters as well. Note that **demand-distance-ratio** $:= \dfrac{demand}{distance \cdot cost}$
4. Merge all 100-demand-clusters into 500-demand-clusters. Calculate demand-distance-ratio. 
5. Merge all 100-demand-clusters into 1000-demand-clusters. Calculate demand-distance-ratio.
6. Choose cluster with best ratio (smallest value) and mark the customers as final. Check if all customers are marked final. 
7. If not check if we still have all types of cars available. Recalculate the clusters for all non-final clusters for all available car types (repeat step 3-5). 

**REMEMBER:** do not hard code parameters, name them 

## 2. Creating coordinate matrix from distance matrix

In [7]:
# input: 101 x 101 matrix (use cleaned_distance_matrix and cleaned_distance_matrix_2)
# depot is node 0 - set it in (0,0)
# output: np.array coordinate_matrix with 2 columns displaying (x,y)-coordinates

## 3. Clustering

In [8]:
# input: distance matrix (use cleaned_distance_matrix and cleaned_distance_matrix_2) and geometrical representation
# of the customers (from point 2), demand_matrix, capacity_matrix

# output: (cluster x customer)-matrix that consists only of 0 and 1 (1 means that the customer is part of the cluster)
# The size of the (cluster x customer)-matrix is approx. 19 x 100
# output: a demand_per_cluster_matrix that consists of the demand per cluster [demand_of_cluster_1, demand_of_cluster_2, ...]

I think after we have our size 100 clusters we should not keep on working with our approximation of distances but use some algorithm to get the "actual" distance of each cluster. since these small clsuter only have few customers this should go rather fast and will yiel much better results when it comes to the merging. we dont need to calculate this new after every mergestep here we will just add the distances of the two clusters and delete 2 times the distance from this new cluster to depot???


In [7]:


def is_empty(any_structure):
    if any_structure:
        
        return False
    else:
        
        return True

In [8]:
def average_linkage_dist_for_size100(cluster, customer):
    i=4
    distance=0
    k=0
    while(i<=len(cluster)-1):
        k+=1
        x=cluster[i]
        #print(x)
        distance+=distance_matrix[x][customer]
        i+=1
        #print(i)
    distance=distance/k
    return distance

In [9]:
def merge(previous_merged_version,cluster):
    merged_cluster=[]
    #print(previous_merged_version[0])
    #print(previous_merged_version[0]+cluster[0])
    #combine demand of merged clusters
    
    merged_cluster.append(previous_merged_version[0]+cluster[0])
    #create new centroid of merged cluster
    merged_cluster.append(placeholder)
    #distance of merged list PLACEHOLDER
    merged_cluster.append(0)
    #demand_cost ratio of merged cluster PLACEHOLDER
    merged_cluster.append(0)
    customers_of_cluster1=previous_merged_version[4:]
    customers_of_cluster2=cluster[4:]
    merged_cluster= merged_cluster+customers_of_cluster1+customers_of_cluster2
    #print(merged_cluster)
    #distance of merged list
    distance_approximation([merged_cluster])
    #print(merged_cluster)
    #demand_cost ratio of merged cluster
    demand_cost_ratio([merged_cluster])
    #print(merged_cluster)
    return merged_cluster
    
#merge([94, (4.0, 5.8), 38.091131963046834, 0.2741962211723417, 2, 6, 5, 1, 8],[73, (2.8333333333333335, 4.0), 22.803627446568495, 0.3556938969519815, 3, 4, 7])    

In [10]:
def combined_ratio(list_of_clusters):
    if isinstance(list_of_clusters[0], int):
        return
    combinedratio=0
    counter=0
    overall_demand=0
    for list in list_of_clusters:
        counter +=1
        overall_demand+=list[0]
        combinedratio += list[3]*list[0]
    #print('number of clusters')
    #print(counter)
    #print('Combined Ratio')
    return(combinedratio/overall_demand)
    
    

In [11]:
def cost(list_of_clusters):
    cost=0
    for list in list_of_clusters:
        if list[0] > 500:
            cost+=(list[2]*18)
            
        elif list[0] > 300:
            cost+=(list[2]*15)
            
        elif list[0] > 100:
            cost+=(list[2]*13)
            
        else:
            cost+=(list[2]*9)
    return cost
        
    


In [12]:
def demand_cost_ratio(clusters):
    for list in clusters:
        if list[0] > 500:
            list[3]=list[0]/(list[2]*18)
            
        elif list[0] > 300:
            list[3]=list[0]/(list[2]*15)
            
        elif list[0] > 100:
            list[3]=list[0]/(list[2]*13)
            
        else:
            list[3]=list[0]/(list[2]*9)
    return clusters
            


In [13]:
def distance_approximation(clusters):
    #print(clusters)
    for list in clusters:
        
        customer_list= list[4:]
        
        customer_list2= list[5:]
        
        
        approx_dist=0
        counter=0
        for k in range(len(customer_list)):
            counter +=1
            
            for j in range(len(customer_list2)):
                
                approx_dist=approx_dist + distance_matrix[customer_list[k]][customer_list2[j]]
        average_dist_to_depot=average_linkage_dist_for_size100(list, 0)
        approx_dist= ((approx_dist/counter)*0.85+2*average_dist_to_depot)
        list[2]= approx_dist


In [ ]:
def constraint(capacity_matrix):
    number = -1
    counter
    cars_constraint = []
    for i in range(len(capacity_matrix)):
        if number != capacity_matrix[i]
           counter += 1
        else
            cars_constraint.append(counter)
            counter = 0
    return cars_constraint

In [ ]:
constraint(capacity_matrix)

In [ ]:
def how_many_cars(solution):
    car1=0
    car2=0
    car3=0
    car4=0
    for list in solution:
        if list[0]>500:
            car4+=1
        elif list[0]>300:
            car3+=1
        elif list[0]>100:
            car2+=1
        else:
            car1+=1
            
    cars_count = [car1,car2,car3,car4]
    
    return cars_count        

In [48]:
def check_car_constraint(cars_count,constraint):
    constraint_fulfilled = True
    for i in range(len(cars_count)):
        if (cars_count[i]-constraint[i]) > 0:
            constraint_fulfilled = False
            
    return constraint_fulfilled

In [14]:
from scipy.spatial import distance
import operator
import copy

customer_cnt= 100

#Get the euclidean distance
# INSTEAD maybe use the average distance from depot and back instead of euclidian distance
#
#Probably better to use average distancce instead of euclidain distance
#
#
list_of_dist_to_depot=[]
for i in range((customer_cnt)):
    list_of_dist_to_depot.append(distance_matrix[i+1][0])

#print("list distances from customers to depot")
#print(list_of_dist_to_depot)

customer_nodes=[]

#Get the indices
for i in range(customer_cnt):
    customer_nodes.append(i+1)

#create dictionary with customer indices and corresponding distance to depot
customer_nodes_with_dist_to_depot = dict(zip(customer_nodes,list_of_dist_to_depot))

print()
print("customer numbers with distance to depot")
print(customer_nodes_with_dist_to_depot)


demand = [24, 21, 11, 28, 21, 20, 13, 12, 27, 27, 24, 22, 14, 22, 14, 16, 25, 28, 11, 22, 23, 18, 23, 18, 15, 22, 12, 17, 13, 12, 23, 28, 26, 14, 29, 11, 29, 19, 23, 11, 15, 27, 19, 23, 22, 29, 12, 13, 28, 14, 14, 23, 11, 14, 22, 23, 17, 25, 23, 15, 13, 20, 29, 21, 28, 21, 24, 27, 10, 21, 11, 27, 28, 10, 16, 23, 27, 10, 23, 16, 17, 23, 14, 28, 29, 16, 20, 10, 18, 12, 14, 18, 15, 10, 11, 14, 28, 10, 20, 21]
customer_nodes_with_demand = dict(zip(customer_nodes,demand))
#print("customer_nodes_with_demand")
#print(customer_nodes_with_demand)
print()


list_of_clusters = []
copy_of_customer_nodes=customer_nodes[:]
copy_of_customer_nodes_with_dist_to_depot = copy.deepcopy(customer_nodes_with_dist_to_depot)

#this just exist because we used to work with centroids and I didnt want to change abunch of code again
placeholder= 'placeholder'

while True:
    
    cluster=[]

    demand=0

    #choose customer farthes away from depot to create cluster
    cust = max(copy_of_customer_nodes_with_dist_to_depot.items(), key=operator.itemgetter(1))[0]
    del copy_of_customer_nodes_with_dist_to_depot[cust]
    copy_of_customer_nodes[cust-1]=0
    
    

    demand_cust = customer_nodes_with_demand.get(cust)
    
    #at position 0 of the cluster we save the demand of the cluster
    cluster.append(demand_cust)
    #at position 1 of the cluster we save the centroid of the cluster
    cluster.append(placeholder)
    #at position 2 we save the approximated distance of the cluster
    cluster.append(0)
    #at position 3 we save the demand/(distance*gasolineusage) Ratio
    cluster.append(0)
    #from position 4 onwards we save the customer numbers
    cluster.append(cust)
    
    
    demand=demand_cust

    number_of_customers_in_cluster=1


    #add customers with smalles demand/distance ratio to  cluster as long as overall demand of cluster is <= 100
    while True:
        best_demand_distance_ratio=0
        cust_index = -1
        print(copy_of_customer_nodes)
        #find 'nearest' customer to cluster
        for i in range(len(copy_of_customer_nodes)):
            cust_index += 1
            print(copy_of_customer_nodes[cust_index])
            print("cust_index")
            print(cust_index)
            if copy_of_customer_nodes[cust_index] != 0:
                average_distance = average_linkage_dist_for_size100(cluster,copy_of_customer_nodes[cust_index])
                if average_distance > 0:
                    cluster_copy2= cluster[:]
                    cluster_copy2.append(copy_of_customer_nodes[cust_index])
                    distance_approximation([cluster_copy2])
                    cluster_copy2[0]= cluster_copy2[0]+customer_nodes_with_demand.get(copy_of_customer_nodes[cust_index])
                    demand_distance_ratio=cluster_copy2[0]/cluster_copy2[2]
                    if demand_distance_ratio > best_demand_distance_ratio and average_distance >0 and cluster_copy2[0]<=100:
                        best_demand_distance_ratio=demand_distance_ratio
                        cust_smallest_dist =copy_of_customer_nodes[cust_index]
            
        


        #add customer number to cluster and increase demand
        cluster.append(cust_smallest_dist)
        demand= demand+customer_nodes_with_demand.get(cust_smallest_dist)
        cluster[0]=demand
        
     
        

        #if cluster has a demand larger than 100 delete the last customer added, reduce the demand and stop while loop
        if cluster[0]>100:
            print('cluster too large')
            print(cluster)
            cluster[0]=cluster[0]-customer_nodes_with_demand.get(cluster[-1])
            del cluster[-1]
            break 
            
            
      
        del copy_of_customer_nodes_with_dist_to_depot[copy_of_customer_nodes[cust_smallest_dist-1]]
        copy_of_customer_nodes[cust_smallest_dist-1]=0
        
        number_of_customers_in_cluster +=1
        if is_empty(copy_of_customer_nodes_with_dist_to_depot):
            break
       
    list_of_clusters.append(cluster)
    
    if is_empty(copy_of_customer_nodes_with_dist_to_depot):
        break

distance_approximation(list_of_clusters)
demand_cost_ratio(list_of_clusters)
print('list of clusters with demand ratio')
print(list_of_clusters)

print(len(list_of_clusters))
combined_ratio(list_of_clusters)
    



customer numbers with distance to depot
{1: 58, 2: 126, 3: 436, 4: 403, 5: 276, 6: 316, 7: 48, 8: 157, 9: 10, 10: 60, 11: 329, 12: 480, 13: 24, 14: 178, 15: 73, 16: 197, 17: 214, 18: 328, 19: 181, 20: 251, 21: 265, 22: 183, 23: 189, 24: 165, 25: 189, 26: 83, 27: 107, 28: 147, 29: 379, 30: 326, 31: 150, 32: 266, 33: 429, 34: 340, 35: 138, 36: 155, 37: 208, 38: 133, 39: 42, 40: 117, 41: 127, 42: 252, 43: 114, 44: 377, 45: 279, 46: 160, 47: 274, 48: 343, 49: 239, 50: 234, 51: 391, 52: 279, 53: 59, 54: 60, 55: 13, 56: 249, 57: 283, 58: 8, 59: 399, 60: 261, 61: 173, 62: 105, 63: 385, 64: 131, 65: 294, 66: 154, 67: 115, 68: 483, 69: 180, 70: 302, 71: 384, 72: 228, 73: 477, 74: 301, 75: 429, 76: 49, 77: 301, 78: 209, 79: 310, 80: 232, 81: 74, 82: 291, 83: 245, 84: 226, 85: 406, 86: 160, 87: 142, 88: 235, 89: 187, 90: 287, 91: 66, 92: 296, 93: 325, 94: 34, 95: 80, 96: 428, 97: 241, 98: 196, 99: 406, 100: 36}

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 

29
cust_index
28
30
cust_index
29
31
cust_index
30
32
cust_index
31
33
cust_index
32
34
cust_index
33
0
cust_index
34
36
cust_index
35
0
cust_index
36
38
cust_index
37
39
cust_index
38
40
cust_index
39
41
cust_index
40
42
cust_index
41
43
cust_index
42
44
cust_index
43
45
cust_index
44
46
cust_index
45
47
cust_index
46
48
cust_index
47
49
cust_index
48
50
cust_index
49
51
cust_index
50
52
cust_index
51
53
cust_index
52
54
cust_index
53
55
cust_index
54
56
cust_index
55
57
cust_index
56
0
cust_index
57
59
cust_index
58
60
cust_index
59
61
cust_index
60
62
cust_index
61
63
cust_index
62
64
cust_index
63
65
cust_index
64
66
cust_index
65
67
cust_index
66
0
cust_index
67
69
cust_index
68
70
cust_index
69
71
cust_index
70
72
cust_index
71
73
cust_index
72
74
cust_index
73
75
cust_index
74
76
cust_index
75
77
cust_index
76
78
cust_index
77
79
cust_index
78
80
cust_index
79
81
cust_index
80
82
cust_index
81
83
cust_index
82
84
cust_index
83
85
cust_index
84
86
cust_index
85
87
cust_index
86
8

92
cust_index
91
93
cust_index
92
94
cust_index
93
95
cust_index
94
96
cust_index
95
97
cust_index
96
98
cust_index
97
99
cust_index
98
0
cust_index
99
[1, 2, 0, 4, 5, 6, 7, 0, 0, 0, 11, 0, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 0, 36, 0, 38, 39, 40, 41, 42, 0, 44, 45, 0, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 0, 59, 60, 61, 62, 63, 64, 65, 66, 67, 0, 69, 70, 71, 72, 0, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 0, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 0]
1
cust_index
0
2
cust_index
1
0
cust_index
2
4
cust_index
3
5
cust_index
4
6
cust_index
5
7
cust_index
6
0
cust_index
7
0
cust_index
8
0
cust_index
9
11
cust_index
10
0
cust_index
11
13
cust_index
12
14
cust_index
13
15
cust_index
14
16
cust_index
15
17
cust_index
16
18
cust_index
17
19
cust_index
18
20
cust_index
19
21
cust_index
20
22
cust_index
21
23
cust_index
22
24
cust_index
23
25
cust_index
24
26
cust_index
25
27
cust_index
26
28
cust_index
27
29
cust_inde

cust_index
54
56
cust_index
55
57
cust_index
56
0
cust_index
57
59
cust_index
58
60
cust_index
59
61
cust_index
60
62
cust_index
61
63
cust_index
62
64
cust_index
63
65
cust_index
64
66
cust_index
65
67
cust_index
66
0
cust_index
67
69
cust_index
68
70
cust_index
69
71
cust_index
70
72
cust_index
71
0
cust_index
72
74
cust_index
73
75
cust_index
74
0
cust_index
75
77
cust_index
76
78
cust_index
77
79
cust_index
78
80
cust_index
79
81
cust_index
80
82
cust_index
81
83
cust_index
82
0
cust_index
83
85
cust_index
84
86
cust_index
85
87
cust_index
86
88
cust_index
87
89
cust_index
88
90
cust_index
89
91
cust_index
90
92
cust_index
91
93
cust_index
92
94
cust_index
93
95
cust_index
94
96
cust_index
95
97
cust_index
96
98
cust_index
97
99
cust_index
98
0
cust_index
99
[0, 2, 0, 4, 5, 6, 7, 0, 0, 0, 11, 0, 0, 14, 0, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 0, 34, 0, 36, 0, 38, 0, 40, 41, 42, 0, 44, 45, 0, 47, 48, 49, 50, 51, 52, 53, 54, 0, 56, 57, 0, 59, 60, 61, 62,

17
19
cust_index
18
20
cust_index
19
21
cust_index
20
22
cust_index
21
23
cust_index
22
24
cust_index
23
25
cust_index
24
0
cust_index
25
0
cust_index
26
28
cust_index
27
29
cust_index
28
30
cust_index
29
31
cust_index
30
32
cust_index
31
0
cust_index
32
34
cust_index
33
0
cust_index
34
36
cust_index
35
0
cust_index
36
38
cust_index
37
0
cust_index
38
40
cust_index
39
41
cust_index
40
42
cust_index
41
0
cust_index
42
44
cust_index
43
45
cust_index
44
0
cust_index
45
47
cust_index
46
48
cust_index
47
49
cust_index
48
50
cust_index
49
51
cust_index
50
52
cust_index
51
53
cust_index
52
54
cust_index
53
0
cust_index
54
56
cust_index
55
57
cust_index
56
0
cust_index
57
59
cust_index
58
60
cust_index
59
61
cust_index
60
62
cust_index
61
63
cust_index
62
0
cust_index
63
65
cust_index
64
66
cust_index
65
0
cust_index
66
0
cust_index
67
69
cust_index
68
70
cust_index
69
71
cust_index
70
72
cust_index
71
0
cust_index
72
74
cust_index
73
0
cust_index
74
0
cust_index
75
77
cust_index
76
78
cust_in

...

cust_index
95
0
cust_index
96
0
cust_index
97
0
cust_index
98
0
cust_index
99
cluster too large
[114, 'placeholder', 0, 0, 44, 21, 45, 79, 79]
[0, 0, 0, 0, 0, 6, 7, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 0, 19, 0, 0, 22, 0, 0, 0, 0, 0, 28, 0, 30, 0, 0, 0, 34, 0, 36, 0, 0, 0, 40, 41, 0, 0, 0, 0, 0, 47, 0, 0, 50, 0, 52, 53, 0, 0, 0, 0, 0, 0, 60, 61, 0, 0, 0, 0, 0, 0, 0, 69, 0, 0, 0, 0, 0, 0, 0, 0, 78, 0, 80, 0, 0, 0, 0, 0, 86, 0, 0, 0, 90, 0, 92, 93, 94, 0, 0, 0, 0, 0, 0]
0
cust_index
0
0
cust_index
1
0
cust_index
2
0
cust_index
3
0
cust_index
4
6
cust_index
5
7
cust_index
6
0
cust_index
7
0
cust_index
8
0
cust_index
9
11
cust_index
10
0
cust_index
11
0
cust_index
12
0
cust_index
13
0
cust_index
14
0
cust_index
15
0
cust_index
16
0
cust_index
17
19
cust_index
18
0
cust_index
19
0
cust_index
20
22
cust_index
21
0
cust_index
22
0
cust_index
23
0
cust_index
24
0
cust_index
25
0
cust_index
26
28
cust_index
27
0
cust_index
28
30
cust_index
29
0
cust_index
30
0
cust_index
31
0
cust_index
32
34
cust_in

22
0
cust_index
23
0
cust_index
24
0
cust_index
25
0
cust_index
26
28
cust_index
27
0
cust_index
28
30
cust_index
29
0
cust_index
30
0
cust_index
31
0
cust_index
32
0
cust_index
33
0
cust_index
34
36
cust_index
35
0
cust_index
36
0
cust_index
37
0
cust_index
38
40
cust_index
39
41
cust_index
40
0
cust_index
41
0
cust_index
42
0
cust_index
43
0
cust_index
44
0
cust_index
45
47
cust_index
46
0
cust_index
47
0
cust_index
48
50
cust_index
49
0
cust_index
50
0
cust_index
51
53
cust_index
52
0
cust_index
53
0
cust_index
54
0
cust_index
55
0
cust_index
56
0
cust_index
57
0
cust_index
58
60
cust_index
59
61
cust_index
60
0
cust_index
61
0
cust_index
62
0
cust_index
63
0
cust_index
64
0
cust_index
65
0
cust_index
66
0
cust_index
67
69
cust_index
68
0
cust_index
69
0
cust_index
70
0
cust_index
71
0
cust_index
72
0
cust_index
73
0
cust_index
74
0
cust_index
75
0
cust_index
76
78
cust_index
77
0
cust_index
78
0
cust_index
79
0
cust_index
80
0
cust_index
81
0
cust_index
82
0
cust_index
83
0
cust_in

0
cust_index
87
0
cust_index
88
90
cust_index
89
0
cust_index
90
0
cust_index
91
93
cust_index
92
0
cust_index
93
0
cust_index
94
0
cust_index
95
0
cust_index
96
0
cust_index
97
0
cust_index
98
0
cust_index
99
[0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 22, 0, 0, 0, 0, 0, 0, 0, 30, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 41, 0, 0, 0, 0, 0, 47, 0, 0, 50, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60, 61, 0, 0, 0, 0, 0, 0, 0, 69, 0, 0, 0, 0, 0, 0, 0, 0, 78, 0, 0, 0, 0, 0, 0, 0, 86, 0, 0, 0, 90, 0, 0, 93, 0, 0, 0, 0, 0, 0, 0]
0
cust_index
0
0
cust_index
1
0
cust_index
2
0
cust_index
3
0
cust_index
4
6
cust_index
5
0
cust_index
6
0
cust_index
7
0
cust_index
8
0
cust_index
9
0
cust_index
10
0
cust_index
11
0
cust_index
12
0
cust_index
13
0
cust_index
14
0
cust_index
15
0
cust_index
16
0
cust_index
17
0
cust_index
18
0
cust_index
19
0
cust_index
20
22
cust_index
21
0
cust_index
22
0
cust_index
23
0
cust_index
24
0
cust_index
25
0
cust_index
26
0
cust_index
27
0
cust_index
28
30
cust_index
29
0
cus

15
0
cust_index
16
0
cust_index
17
0
cust_index
18
0
cust_index
19
0
cust_index
20
0
cust_index
21
0
cust_index
22
0
cust_index
23
0
cust_index
24
0
cust_index
25
0
cust_index
26
0
cust_index
27
0
cust_index
28
0
cust_index
29
0
cust_index
30
0
cust_index
31
0
cust_index
32
0
cust_index
33
0
cust_index
34
0
cust_index
35
0
cust_index
36
0
cust_index
37
0
cust_index
38
0
cust_index
39
0
cust_index
40
0
cust_index
41
0
cust_index
42
0
cust_index
43
0
cust_index
44
0
cust_index
45
47
cust_index
46
0
cust_index
47
0
cust_index
48
0
cust_index
49
0
cust_index
50
0
cust_index
51
0
cust_index
52
0
cust_index
53
0
cust_index
54
0
cust_index
55
0
cust_index
56
0
cust_index
57
0
cust_index
58
60
cust_index
59
0
cust_index
60
0
cust_index
61
0
cust_index
62
0
cust_index
63
0
cust_index
64
0
cust_index
65
0
cust_index
66
0
cust_index
67
0
cust_index
68
0
cust_index
69
0
cust_index
70
0
cust_index
71
0
cust_index
72
0
cust_index
73
0
cust_index
74
0
cust_index
75
0
cust_index
76
78
cust_index
77
0


0.014610306977961757

In [44]:
def spread_out_or_weed_out(solution,list_of_original_clusters,scaler):


   
    
    new_original_merge=solution
    population=[]

    
    if isinstance(solution[0], int):
        print('SOLUTION')
        print(solution)
        return
   

    k1=0
    threshold=combined_ratio(list_of_original_clusters)*scaler

    best_solution = combined_ratio(list_of_original_clusters)
    
    for i in range(len(solution)):
        
        k2=k1

        for j in range(len(solution)):
            cluster_list_to_work=solution[:]
            k2+=1
            if k2==len(solution):
                break
            mergedversion=merge(solution[k1],solution[k2])
            del cluster_list_to_work[k1]
            del cluster_list_to_work[k2-1]
            cluster_list_to_work.append(mergedversion)
            cr_merge=combined_ratio(cluster_list_to_work)
            if cr_merge > best_solution:
                best_solution = cr_merge
            
            if cr_merge>threshold and mergedversion[0] < 1000:
                population.append(cluster_list_to_work)
        k1+=1


    if is_empty(population):
        #print('population is empty')
        return solution,best_solution
    return population,best_solution


First reduce search space by merging those size 100 clusters that obviously belong together.

after we have our list of size 100 custers we create from this list a population of solution, 
in which a solution is our initial list of size 100 clusters with 2 random clusters havin been merged.
the only criteris each solution has to fullfill to become part of the population is, that the merged cluster
has no demand higher than 1000 and the solution has a combined demand/cost ratio that is not smaller than the demand/cost ratio 
of the original list of size 100 custers multiplied with a scaler.
this scaler is very important: if the scaler is smaller than 1 that means we allow solutions in our population that are worse than
the original list of size 100 custers. this allows us to make steps that are in the shortrun bad but could lead in the longrun to better
results. we explore and spread out our solution.
if the scaler is larger than 1 we weed out all the solutions that are worse than the original list of size 100 custers.

what we need to do is have the scaler switch between below 1 and above 1. we want it to be below 1 more often than above but we also want it
to increase in size when it is above 1 so that we weed out more an more weak solutions. and in the longrund get better and better solutions.



In [ ]:
from scipy.spatial import distance
import operator
import copy
import random
list_of_clusters=[[100, 'placeholder', 603.6125, 0.018407688891650044, 68, 9, 58, 100], [99, 'placeholder', 764.95, 0.01438002483822472, 12, 35, 37, 43], [97, 'placeholder', 776.05, 0.013887994043911833, 73, 46, 84, 8], [99, 'placeholder', 670.1600000000001, 0.016413990688790736, 3, 10, 1, 39, 15], [99, 'placeholder', 589.4, 0.018663047166610113, 33, 55, 76, 13, 91], [95, 'placeholder', 659.14, 0.016014132893703244, 75, 26, 67, 64, 27], [96, 'placeholder', 727.2875, 0.014666368756051307, 96, 97, 72, 42], [96, 'placeholder', 745.4875, 0.014308310557409302, 85, 17, 32, 83], [99, 'placeholder', 831.38, 0.013231013495633765, 99, 49, 89, 56, 88], [100, 'placeholder', 671.21, 0.01655385216416786, 4, 2, 62, 66, 98], [100, 'placeholder', 705.5699999999999, 0.015747709102018385, 59, 23, 87, 24, 16], [100, 'placeholder', 960.95, 0.011562631886269952, 51, 77, 65, 5, 74], [94, 'placeholder', 431.35, 0.024213386911891607, 63, 81, 31, 54, 95], [100, 'placeholder', 749.1, 0.014832613951556682, 71, 18, 82, 70, 57], [91, 'placeholder', 788.77, 0.012818833260787189, 29, 14, 38, 25, 20], [91, 'placeholder', 720.2625, 0.01403809182223302, 44, 21, 45, 79], [94, 'placeholder', 955.9100000000001, 0.010926179707759562, 48, 11, 52, 80, 92], [98, 'placeholder', 1001.675, 0.010870680499052977, 34, 7, 94, 53, 40, 19, 28, 36], [98, 'placeholder', 883.8928571428571, 0.012319240732509956, 30, 41, 22, 86, 61, 69, 50], [84, 'placeholder', 1281.9583333333335, 0.0072805278382682736, 93, 6, 47, 78, 60, 90]]
print(combined_ratio(list_of_clusters))
print()
original_clusters=list_of_clusters

'''
#in the first step we set the scaler high to get a small population to start with, by only allowing the most obvious merges
scaler=0.94
population,best_solution=spread_out_or_weed_out(list_of_clusters,original_clusters,scaler)
#in the next step we set the scaler low to increase our population by allowing worse solutions in the population than 
#the original list of clusters

#while population==list_of_clusters:
#    scaler=scaler-0.00001
#    population=spread_out_or_weed_out(list_of_clusters,original_clusters,scaler)

#print(len(population))
#if len(population)<3:
#    scaler=scaler
#elif len(population)<9:
#    scaler=scaler
#else:
#    scaler=scaler+0.01

all_best_solutions = []
    
new_population = []     
scaler=0.97

print(len(population))

for list in population:
    solution=list
    #print(solution)
    descendants_of_solution,best_solution=spread_out_or_weed_out(solution,original_clusters,scaler)
    all_best_solutions.append(best_solution)
    if descendants_of_solution != solution:
        if len(descendants_of_solution)==1:
            new_population.append(descendants_of_solution[0])
        else:
            for list in descendants_of_solution:
                #population2=list
                #for list in population2:
                #print(combined_ratio(list))
                new_population.append(list)

population = new_population            
new_population = []     
scaler=0.97

print(len(population))

for list in population:
    solution=list
    #print(solution)
    descendants_of_solution,best_solution=spread_out_or_weed_out(solution,original_clusters,scaler)
    all_best_solutions.append(best_solution)
    if descendants_of_solution != solution:
        if len(descendants_of_solution)==1:
            new_population.append(descendants_of_solution[0])
        else:
            for list in descendants_of_solution:
                #population2=list
                #for list in population2:
                #print(combined_ratio(list))
                new_population.append(list)


population = new_population
new_population = []    
scaler=0.97
    
print(len(population))

for list in population:
    solution=list
    #print(solution)
    descendants_of_solution,best_solution=spread_out_or_weed_out(solution,original_clusters,scaler)
    all_best_solutions.append(best_solution)
    if descendants_of_solution != solution:
        if len(descendants_of_solution)==1:
            new_population.append(descendants_of_solution[0])
        else:
            for list in descendants_of_solution:
                #population2=list
                #for list in population2:
                #print(combined_ratio(list))
                new_population.append(list)
                    

population = new_population
new_population = []
scaler=0.98
    
print(len(population))

for list in population:
    solution=list
    #print(solution)
    descendants_of_solution,best_solution=spread_out_or_weed_out(solution,original_clusters,scaler)
    all_best_solutions.append(best_solution)
    if descendants_of_solution != solution:
        if len(descendants_of_solution)==len(solution)-1:
            new_population.append(descendants_of_solution[0])
        else:
            for list in descendants_of_solution:
                #population2=list
                #for list in population2:
                #print(combined_ratio(list))
                new_population.append(list)
            
population=new_population

new_population = []
scaler=0.97
    
print(len(population))

for list in population:
    solution=list
    #print(solution)
    descendants_of_solution,best_solution=spread_out_or_weed_out(solution,original_clusters,scaler)
    all_best_solutions.append(best_solution)
    if descendants_of_solution != solution:
        if len(descendants_of_solution)==len(solution)-1:
            new_population.append(descendants_of_solution[0])
        else:
            for list in descendants_of_solution:
                #population2=list
                #for list in population2:
                #print(combined_ratio(list))
                new_population.append(list)
            
population=new_population


new_population = []
scaler=0.99
    
print(len(population))

for list in population:
    solution=list
    #print(solution)
    descendants_of_solution,best_solution=spread_out_or_weed_out(solution,original_clusters,scaler)
    if descendants_of_solution != solution:
        if len(descendants_of_solution)==len(solution)-1:
            new_population.append(descendants_of_solution[0])
        else:
            for list in descendants_of_solution:
                #population2=list
                #for list in population2:
                #print(combined_ratio(list))
                new_population.append(list)
            
population=new_population

#from every solution in the population we create a new population and save it in the list of populations
print(len(all_best_solutions))
print('fjdhfzwefhe')
print(len(population))
print(population)


#population=new_population

#print(combined_ratio(population))
#for list in population:
   # print(combined_ratio(list))
'''

all_best_solutions = []
scaler= 0.95
population,best_solution=spread_out_or_weed_out(list_of_clusters,original_clusters,scaler)

for i in range(30):
    new_population = []     
    print(len(population))

    for list in population:
        solution=list
        descendants_of_solution,best_solution=spread_out_or_weed_out(solution,original_clusters,scaler)
        all_best_solutions.append(best_solution)
        if descendants_of_solution != solution:
            if len(descendants_of_solution)==1:
                new_population.append(descendants_of_solution[0])
            else:
                for list in descendants_of_solution:
                    new_population.append(list)
                    

    if len(population) > 10000:
        scaler+=0.01
    elif len(population) > 1000:
        scaler+=0.001
    elif len(population) > 100:
        scaler+=0.0001
    else:
        scaler=scaler
    population = new_population            
  



0.014610306977961757

163
4234
30211
75019


## 4. Merging

In [ ]:
# input: (cluster x customer)-matrix, demand_per_cluster_matrix, demand_matrix, cleaned_distance_matrix, 
# capacity_matrix, coordinate_matrix

# output: final_cluster_customer_matrix

## 5. Differential Evolution/Genetic Algorithm/Ant Algorithm

In [ ]:
# input: final_cluster_customer_matrix

## 6. Run